* csv ver

In [1]:
# 임포트
import json
import time
import random
import requests
import psycopg2
import datetime
import numpy  as np
import pandas as pd
from tqdm import tqdm
from fake_useragent import UserAgent

ua = UserAgent()

### 1. 깃에서 법정동 CSV 파일 불러오기 (법정동코드 10자리)
cortar = pd.read_csv('C:\\oasis\\oasisbusiness-ds\\oasisbusiness-ds\\1_데이터수집\\법정동코드전체.csv')
cortar = cortar[cortar['폐지여부\r\n']=='존재\r\n'] # 존재하는 지역만 남기기
lawd_cd_unique = cortar.drop_duplicates('법정동코드')['법정동코드'] # 법정동 코드만 담기 (총 20551개)

In [ ]:
### 2. 전월세 호가 수집
# 변수 생성
fails = [] # 수집 안 된 법정동
info_df = pd.DataFrame() # 최종 데이터프레임

# json to dataframe (법정동 단위 수집)
for i in tqdm(range(0,5000)):
    
    # url 설정
    url = 'https://m.land.naver.com/cluster/ajax/articleList?itemId=&mapKey=&lgeo=&showR0=&rletTpCd=SG&tradTpCd=A1%3AB1%3AB2&z=14&totCnt=402&cortarNo={}&sort=rank'.format(lawd_cd_unique[i])
      
    try:
        # url 호출
        headers = {'user-agent' : ua.ie,'referer' : 'https://m.land.naver.com/'}
        time.sleep(random.randrange(3,7))
        response = requests.get(url, headers = headers)

        # body 태그 선택
        items = response.json()['body']

        # 데이터프레임화
        df = pd.DataFrame(items)
        info_df = info_df.append(df)

    except:
        fails.append([i])
    
    # 받아온 바로 적재 (언제 막힐지 모르니까 바로바로 적재함)
    if len(info_df) > 0:
        
        # 컬럼 정리
        info_df = info_df[['atclNo','cortarNo','atclNm','atclStatCd','rletTpCd','uprRletTpCd','rletTpNm','tradTpCd','tradTpNm','vrfcTpCd','flrInfo','prc','rentPrc','spc1','spc2','direction','atclCfmYmd','lat','lng','atclFetrDesc','cpid','cpNm','rltrNm']]
        # NaN 처리
        info_df = info_df.fillna(np.nan)

        ### 3. DB INSERT
        df = info_df
        table = 'm1.asking_price'

        # execute_mogrify
        def execute_mogrify(conn, df, table, val):
            # Create a list of tuples from the dataframe values
            tuples = [tuple(x) for x in df.to_numpy()]
            # Comma-separated dataframe columns
            cols = ','.join(list(df.columns))
            # SQL query to execute
            cursor = conn.cursor()
            values = [cursor.mogrify(val, tup).decode('utf8') for tup in
                      tuples]
            query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

            try:
                cursor.execute(query, tuples)
                conn.commit()
            except (Exception, psycopg2.DatabaseError) as error:
                print("Error: %s" % error)
                conn.rollback()
                cursor.close()
                return 1
            cursor.close()

        # values 설정
        val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

        # insert
        for j in range(0, len(df), 10000):

            # DB Connect
            conn = psycopg2.connect(
                host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
                port=5439,
                dbname='dev',
                user='awsuser',
                password='cremaoAdmin1234qwer!!'
            )

            # 10000개 단위로 나누기
            tmp = df[j:j + 10000]

            # insert
            execute_mogrify(conn, tmp, table, val)
            print(datetime.datetime.now(), ' : ', j)
            tmp = pd.DataFrame()

        # row count
        print(len(df))

  0%|                                                                                                          | 2/5000 [00:17<12:16:36,  8.84s/it]

2022-10-06 18:55:44.312252  :  0
20


  0%|                                                                                                          | 3/5000 [00:25<11:53:18,  8.56s/it]

2022-10-06 18:55:52.561799  :  0
24


  0%|                                                                                                          | 4/5000 [00:35<12:49:24,  9.24s/it]

...

2022-10-06 19:01:05.907768  :  0
322


  1%|▊                                                                                                        | 36/5000 [05:47<14:25:34, 10.46s/it]

2022-10-06 19:01:14.492961  :  0
327


  1%|▊                                                                                                        | 37/5000 [05:53<12:39:36,  9.18s/it]

2022-10-06 19:01:20.690988  :  0
347
